Hello All. This is my first entry on Kaggle. Therefore I want to focus on readable code and doing a simple analysis with this dataset.

Let's first import the modules we need

In [ ]:
#######################
# Simple analysis of home advantage for every league
# Lars Tijssen | lars.tijssen@essl.org
# database: European Soccer League
# 2017-05-31
#######################

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3

Load the database into python

In [ ]:
# load data
path = '../input/database.sqlite'
with sqlite3.connect(path) as con:
    countries = pd.read_sql_query("SELECT * from Country", con)
    matches = pd.read_sql_query("SELECT * from Match", con)
    leagues = pd.read_sql_query("SELECT * from League", con)
    teams = pd.read_sql_query("SELECT * from Team", con)

Let's rework the data a little bit. Especially replacing the id of the League, country and teams columns to their actual names helps a lot in the later run. Also converting the date column to datetime is something that I like because you can then select by date if need be (not really necessary for the purpose of this script though)

In [ ]:
# Sanitize the data

## select the id-column as the index of the dataframe
matches = matches.set_index('id')

## map the actual name to the id 
# league
league_id_map = leagues.set_index('id').to_dict()['name']
matches['league_id'] = matches['league_id'].map(league_id_map)

# country
country_id_map = countries.set_index('id').to_dict()['name']
matches['country_id'] = matches['country_id'].map(country_id_map)

# teams
team_id_map = teams.set_index('team_api_id').to_dict()['team_long_name']
matches['home_team_api_id'] = matches['home_team_api_id'].map(team_id_map)
matches['away_team_api_id'] = matches['away_team_api_id'].map(team_id_map)

## convert the date column to python datetime
matches.date = pd.to_datetime(matches.date)

Okay. Now that we have sanitized the data a little bit, let's see if we can do some analysis. There is a lot of information in these databases so I want to focus on something simple. Let's see if we can see what the ratio is between home goals and away goals for every individual league. Since this is an average stat we don't have to worry about the amount of teams or amount of games played in every league.

First we select the **home_team_goal** and **away_team_goal** columns that we need to calculate these stats and then calculate the mean over these columns for every league using the groupby command.

In [ ]:
# calculate the mean goals per league
mean_goals = matches[['league_id','home_team_goal','away_team_goal']].groupby('league_id').mean()

So now we have the mean goals scored by the home team and away team for every league. To calculate the percentage we simply divide these columns by the sum of the average and multiply by 100.

In [ ]:
# calculate the mean goals per league as percentage from the total
total_goals = mean_goals['home_team_goal'] + mean_goals['away_team_goal']
mean_goals['home_team_goal_percent'] = 1e2 * mean_goals['home_team_goal'] / total_goals
mean_goals['away_team_goal_percent'] = 1e2 * mean_goals['away_team_goal'] / total_goals

# make a subset of 
mean_goals_percent = mean_goals[['home_team_goal_percent','away_team_goal_percent']]

Now that we have done the calculation we can display the pandas dataframe below

In [ ]:
mean_goals_percent

and subsequently show it as a barplot using pandas and matplotlib

In [ ]:
#### plot the mean goals per league as a horizontal bar chart using pandas plotting
mgp_plot = mean_goals_percent.plot(kind='barh', stacked=True, legend = False,
                                   fontsize = 12, width = 0.8, xticks = np.arange(0,100.1,10))

mgp_plot.set_xlabel('Percent', fontsize = 14)
mgp_plot.set_ylabel('League', fontsize = 14)
mgp_plot.set_title('Ratio between home goals and away goals for each League in percent', fontsize = 16)

mgp_plot.vlines(50, ymin = -0.5, ymax = 10.5, linewidth = 0.5, linestyle = '--')

# plot numbers in plot
k = 0
for index, row in mean_goals_percent.iterrows():
    
    k1 = row['home_team_goal_percent'].round(2)
    k2 = row['away_team_goal_percent'].round(2)
    
    mgp_plot.text(x = 25, y = k, s = k1, color = 'white', fontsize = 14, ha = 'center', va = 'center')
    mgp_plot.text(x = 80, y = k, s = k2, color = 'white', fontsize = 14, ha = 'center', va = 'center')
    k += 1
    
plt.legend(bbox_to_anchor=(1.45,1), fontsize = 12)

OK. That's it for now. We can see that in every league the home team has a distinct advantage, with Scotlands teams having the least and Spains teams having the most. Interesting questions to answer would be why this home advantage manifests itself and why the differences between the leagues occur, but that is beyond the scope of this notebook.